# COGS 108 - Data Checkpoint

# Names

- Tyler Le
- Aditya Tomar
- William Lynch
- Michael Mao
- Natalie Quach

<a id='research_question'></a>
# Research Question

*Fill in your research question here*

# Dataset(s)

*Fill in your dataset information here*
**Template**
- Dataset Name:
- Link to the dataset: 
- Number of observations: 

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.



**1. Fatalities and Injuries.**
- Dataset Name: 2020 Summary of Hazardous Weather Fatalities, Injuries, and Damage Costs by State
- Link to the dataset: https://www.weather.gov/hazstat/
- Number of observations: 67 (per year). Each year is a separate dataset.

This is a dataset describing the number of fatalities, injuries, property damage costs, and total damage costs due to Hazardous weather for the year of 2015.

This dataset is for a single year, we plan to gather information for years from XXXX-XXXX and will merge all of them into one.


**2. Frequency of Disasters By State**
- Dataset Name: Billion-Dollar Disasters By Year (CPI-Adjusted)
- Link to the dataset: https://www.ncdc.noaa.gov/billions/state-freq-data.csv
- Number of observations: 2228 

This dataset contains the count of natural disasters that cost more than 1 billion dollars for each state from 1980 - 2021.



**3. Types of Disaster By State**
- Dataset Name: Disaster Declarations Summaries
- Link to the dataset: https://www.fema.gov/api/open/v1/DisasterDeclarationsSummaries.csv
- Number of observations: 62772

1-2 sentences describing each dataset. 
This dataset contains all federally declared natural disasters from 1953-2022 (by year,state, and type) along with declared recovery programs.

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

**4. National Risk Index (NRI)**
- Dataset Name:
- Link to the dataset: https://hazards.fema.gov/nri/data-resources#csvDownload
- Number of observations: 3142

Dataset from FEMA that identifies counties and states most at risk to 18 natural hazards. Includes data about expected annual losses from natural hazards, social vulnerability and community resilience. 



# Setup

In [166]:
import pandas as pd
import os

# Data Cleaning

Describe your data cleaning steps here.

## Clean Table #1 (Fatalaties, Injuries, Damage Costs for 2020 Only)

In [184]:
disaster_2020 = pd.read_csv('datasets/nat_disast_bystate_deaths_cost/state20.csv')
disaster_2020 = disaster_2020.rename(columns={"2020 Summary of Hazardous Weather Fatalities, Injuries, and Damage Costs by State":"state",
                                              "Unnamed: 1": "fatalities", 
                                              "Unnamed: 2": "injuries", 
                                              "Unnamed: 3": "Property Damage (million $)", 
                                              "Unnamed: 4":"Crop Damage (million $)", 
                                              "Unnamed: 5":"Total Damage (million $)"}).iloc[3:-1]
disaster_2020['year'] = 2020
assert(disaster_2020.isna().sum().sum() == 0)

disaster_2020.set_index('year')

#Use the code below to join several dataframes from several years and set index by year

# list_dfs = []

# for csv_file in os.listdir("datasets/nat_disast_bystate_deaths_cost/"):
#     if csv_file.endswith(".csv"):
#         list_dfs.append(pd.read_csv("datasets/nat_disast_bystate_deaths_cost/{0}".format(csv_file)))
# #         result_df = pd.concat(pd.read_csv("datasets/nat_disast_bystate_deaths_cost/{0}".format(csv_file)))

# # disaster_2020
# type(list_dfs[0])
# result = pd.concat(list_dfs)
# result

,state,fatalities,injuries,Property Damage (million $),Crop Damage (million $),Total Damage (million $)
year,,,,,,
2020,AK (Alaska),5,0,35.85,0,35.85
2020,AL (Alabama),11,27,313.39,2.08,315.47
2020,AM (Atlantic South),0,0,0,0,0
2020,AN (Atlantic North),0,0,0.01,0,0.01
2020,AR (Arkansas),2,32,313.81,1.41,315.21
...,...,...,...,...,...,...
2020,VT (Vermont),0,0,1.91,0.01,1.92
2020,WA (Washington),5,11,1340.16,0.52,1340.68
2020,WI (Wisconsin),11,1,79.28,0.26,79.54


## Clean Table #3 (Types of Disaster By State)

In [163]:
disaster_type_df = pd.read_csv('datasets/DisasterDeclarationsSummaries.csv')

wanted_columns = ['state', 'declarationDate','fyDeclared','incidentType','declarationTitle','designatedArea']

disaster_type_df = disaster_type_df[wanted_columns].rename(columns={"fyDeclared": "fiscalYearDeclared", "designatedArea": "area"})
assert(disaster_type_df.isna().sum().sum() == 0)

# disaster_type_df

## Clean Table #4 (NRI)

In [185]:
# Look at the "NRI Data Dictionary in the datasets/NRI_Table_Counties" to see what the cols mean
# EAL = "Expected Annual Lost", quantifies the anticipated economic damage resulting from natural hazards each year. 
# 1-100 scale

df_nri = pd.read_csv('datasets/NRI_Table_Counties/NRI_Table_Counties.csv')
wanted_cols = ['STATE','STATEABBRV','COUNTY','POPULATION','AREA','RISK_SCORE','RISK_RATNG','EAL_SCORE','EAL_RATNG']

df_nri = df_nri[wanted_cols]
assert(df_nri.isna().sum().sum() == 0)

df_nri

,STATE,STATEABBRV,COUNTY,POPULATION,AREA,RISK_SCORE,RISK_RATNG,EAL_SCORE,EAL_RATNG
0,Kentucky,KY,Johnson,23356,261.958144,9.281419,Relatively Low,11.921944,Relatively Low
1,Kentucky,KY,Kenton,159720,160.213975,10.449057,Relatively Low,16.837131,Relatively Moderate
2,Kentucky,KY,Knott,16346,351.517978,10.068395,Relatively Low,10.945913,Relatively Low
3,Kentucky,KY,Knox,31883,386.298435,11.858245,Relatively Low,11.983719,Relatively Low
4,Kentucky,KY,Larue,14193,261.539564,4.610900,Very Low,7.028611,Very Low
...,...,...,...,...,...,...,...,...,...
3137,Wyoming,WY,Sweetwater,43806,10426.919825,2.572885,Very Low,7.174819,Very Low
3138,Wyoming,WY,Teton,21294,3996.855337,6.513988,Very Low,15.004361,Relatively Low
3139,Wyoming,WY,Uinta,21118,2081.651840,4.458528,Very Low,8.437977,Very Low
3140,Wyoming,WY,Washakie,8533,2238.665800,5.182977,Very Low,5.592578,Very Low
